In [1]:
import json
import spacy
from bs4 import BeautifulSoup
nlp = spacy.load("output/model-best")

ModuleNotFoundError: No module named 'bs4'

In [2]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

In [3]:
with open('./texts_for_testing/1880_90030.json', encoding="utf-8") as f:
    test_text = json.load(f)
opinion_of_text = test_text['html_lawbox']
cleaned_text_opinion = strip_html_tags(opinion_of_text)
print(cleaned_text_opinion[:500])


100 U.S. 158 (____)
Dow
v.
JOHNSON.
Supreme Court of United States.

The case was argued by The Attorney-General and Mr. E.B. Smith, Assistant Attorney-General, for the plaintiff in error, and by Mr. Thomas J. Durant for the defendant in error.
MR. JUSTICE FIELD delivered the opinion of the court.
The defendant in the court below, the plaintiff in error here, Neal Dow, was a brigadier-general in the army of the United States during the late civil war, and in 1862 and 1863 was *159 stationed in 


In [4]:
spacy_doc = nlp(cleaned_text_opinion)

In [5]:
spacy_doc.cats

{'1700': 0.0, '1800': 1.0, '1900': 0.0, '2000': 0.0}

In [9]:
with open('./texts_for_testing/2022_opinion_1.json', encoding="utf-8") as f:
    test_text = json.load(f)
opinion_of_text_2 = test_text['plain_text']
spacy_doc_1 = nlp(opinion_of_text_2)
print(spacy_doc_1.cats)
spacy_doc_1.cats['1700']

{'1700': 2.396220373995437e-43, '1800': 2.923387118178614e-29, '1900': 1.0, '2000': 7.380723387276827e-21}


2.396220373995437e-43

In [11]:
with open('./texts_for_testing/1804_84713.json', encoding="utf-8") as f:
    test_text = json.load(f)
opinion_of_text_3 = test_text['html_with_citations']
cleaned_text_opinion = strip_html_tags(opinion_of_text_3)
spacy_doc_2 = nlp(cleaned_text_opinion)
print(spacy_doc_2.cats)
spacy_doc_2.cats['1800']

{'1700': 3.133987236392244e-10, '1800': 1.0, '1900': 0.0, '2000': 0.0}


1.0

In [12]:
with open('./texts_for_testing/1985_111301.json', encoding="utf-8") as f:
    test_text = json.load(f)
opinion_of_text_4 = test_text['html_lawbox']
cleaned_text_opinion = strip_html_tags(opinion_of_text_4)
spacy_doc_3 = nlp(cleaned_text_opinion)
spacy_doc_3.cats

{'1700': 0.0, '1800': 0.0, '1900': 1.0, '2000': 0.0}

In [18]:
from pathlib import Path
folder_location = './texts_for_testing/'
paths = list(Path(folder_location).rglob('*.json'))
filenames = [k.stem for k in paths]
filenames

['1764_2381788',
 '1764_84586',
 '1764_84587',
 '1783_84599',
 '1783_84600',
 '1804_84713',
 '1850_86508',
 '1865_87621',
 '1880_90030',
 '1898_94785',
 '1902_95542',
 '1920_99495',
 '1944_103915',
 '1963_106601',
 '1985_111301',
 '2022_opinion_1',
 '2022_opinion_2']

In [19]:
with open('./texts_for_testing/1764_84586.json', encoding="utf-8") as f:
    test_text = json.load(f)
opinion_of_text = test_text['html_with_citations']
cleaned_text_opinion = strip_html_tags(opinion_of_text)
spacy_doc_3 = nlp(cleaned_text_opinion)
spacy_doc_3.cats

{'1700': 0.9460902810096741,
 '1800': 0.0035104146227240562,
 '1900': 0.021581759676337242,
 '2000': 0.028817567974328995}

In [20]:
with open('./texts_for_testing/1764_84587.json', encoding="utf-8") as f:
    test_text = json.load(f)
opinion_of_text = test_text['html_with_citations']
cleaned_text_opinion = strip_html_tags(opinion_of_text)
spacy_doc_3 = nlp(cleaned_text_opinion)
spacy_doc_3.cats

{'1700': 1.0,
 '1800': 7.777257948760052e-09,
 '1900': 2.8018092301806703e-16,
 '2000': 9.425196398636101e-20}

In [21]:
with open('./texts_for_testing/1764_2381788.json', encoding="utf-8") as f:
    test_text = json.load(f)
opinion_of_text = test_text['html_with_citations']
cleaned_text_opinion = strip_html_tags(opinion_of_text)
spacy_doc_3 = nlp(cleaned_text_opinion)
spacy_doc_3.cats

{'1700': 0.0, '1800': 0.0, '1900': 0.0, '2000': 0.0}

In [ ]:
scores_1700 = []
scores_1800 = []
scores_1900 = []
scores_2000 = []

for file in filenames:
    with open(filename, encoding="utf-8") as f:
        data = json.load(f)
    
opinion_of_text = test_text['html_with_citations']
cleaned_text_opinion = strip_html_tags(opinion_of_text)
spacy_doc_3 = nlp(cleaned_text_opinion)
spacy_doc_3.cats